### Find Word Group Youtube Link

In [55]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [56]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [57]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 5000  # 28 native word end index

# youtube
sample_num = 10  # 7
time_shift = 0.6

In [58]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
            #df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].tail(sample_num)  # will test
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [59]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")\n
    df_search_result is dataframe and "search_string", "start_time", "end_time", "sentence", "video_id" are its columns
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [60]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [61]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [62]:
path = f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 3/Result/2-Find Word Group Youtube Link"

Path(path).mkdir(parents=True, exist_ok=True)

In [63]:
path_folder_file = "/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Turkish/Talk Time/Talk Time 2/Data/Deployment/Talk Time 3 Master File.xlsx"
sheet = "Sheet4"  # Sheet1
file_ext = "Fourgram"

In [64]:
df_file = pd.read_excel(f"{path_folder_file}", sheet_name=f"{sheet}")
#df_file = pd.read_excel(f"twogram.xlsx")
df_file

,4Gram
0,bana yardım eder misin
1,ben de memnun oldum
2,ben de seni seviyorum
3,bir şey daha var
4,bir şey mi oldu
5,bir şey var mı
6,bir sorun mu var
7,bu da ne böyle
8,bu da ne demek
9,bu da neydi böyle


In [65]:
search_list = df_file.iloc[:,0].to_list()
len(search_list)

30

In [66]:
#disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]
disable_video_id_list = []

In [67]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD1/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [68]:
df_youtube_sentence = df_youtube_sentence[~df_youtube_sentence["video_id"].isin(disable_video_id_list)]
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [69]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [70]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [71]:
df_word_group = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_word_group

,search_string,start_time,end_time,sentence,video_id
0,bana yardım eder misin,2874.896,2877.030,teyzeciğim ne olur bana yardım eder misin,QIF4PAQBRS8
1,bana yardım eder misin,3498.016,3499.649,bana yardım eder misin,Qmn1S9yjxB4
2,bana yardım eder misin,4625.087,4626.288,bana yardım eder misin,SnbDtuHGdqE
3,bana yardım eder misin,76.160,79.780,şu oyuncağı bir türlü kaldıramıyorum bana yard...,LMHUJ60Fsho
4,bana yardım eder misin,922.920,924.920,ya bana yardım eder misin,OBoAMIqiTWU
...,...,...,...,...,...
247,seni görmek çok güzel,2444.900,2451.530,yaşayabilir akımı ben yılmaz merhaba yılmaz se...,cUAQrHbbpZ8
248,seni görmek çok güzel,5998.800,6006.360,sonra yumarım açarsın kontrol sızladı da merha...,1iByM08MvqU
249,seni görmek çok güzel,1054.280,1062.900,merhaba canım nasılsın o gün seni görmek çok g...,Rm6zwwnAVeA
250,seni görmek çok güzel,1091.780,1101.250,abone ol bu okey ses geldi tamam süper evet ar...,AvfpnsqaWw4


In [72]:
df_word_group_time_loc = word_group_time_loc(df_word_group, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,bana yardım eder misin,2875.832878,2877.030000,teyzeciğim ne olur bana yardım eder misin,QIF4PAQBRS8
1,bana yardım eder misin,3498.016000,3499.649000,bana yardım eder misin,Qmn1S9yjxB4
2,bana yardım eder misin,4625.087000,4626.288000,bana yardım eder misin,SnbDtuHGdqE
3,bana yardım eder misin,78.134545,79.450909,şu oyuncağı bir türlü kaldıramıyorum bana yard...,LMHUJ60Fsho
4,bana yardım eder misin,923.080000,924.920000,ya bana yardım eder misin,OBoAMIqiTWU
...,...,...,...,...,...
247,seni görmek çok güzel,2447.779697,2449.320000,yaşayabilir akımı ben yılmaz merhaba yılmaz se...,cUAQrHbbpZ8
248,seni görmek çok güzel,6004.306667,6006.360000,sonra yumarım açarsın kontrol sızladı da merha...,1iByM08MvqU
249,seni görmek çok güzel,1056.717980,1058.720606,merhaba canım nasılsın o gün seni görmek çok g...,Rm6zwwnAVeA
250,seni görmek çok güzel,1098.232088,1100.625604,abone ol bu okey ses geldi tamam süper evet ar...,AvfpnsqaWw4


In [73]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id
0,bana yardım eder misin,2875.232878,2877.630000,teyzeciğim ne olur bana yardım eder misin,QIF4PAQBRS8
1,bana yardım eder misin,3497.416000,3500.249000,bana yardım eder misin,Qmn1S9yjxB4
2,bana yardım eder misin,4624.487000,4626.888000,bana yardım eder misin,SnbDtuHGdqE
3,bana yardım eder misin,77.534545,80.050909,şu oyuncağı bir türlü kaldıramıyorum bana yard...,LMHUJ60Fsho
4,bana yardım eder misin,922.480000,925.520000,ya bana yardım eder misin,OBoAMIqiTWU
...,...,...,...,...,...
247,seni görmek çok güzel,2447.179697,2449.920000,yaşayabilir akımı ben yılmaz merhaba yılmaz se...,cUAQrHbbpZ8
248,seni görmek çok güzel,6003.706667,6006.960000,sonra yumarım açarsın kontrol sızladı da merha...,1iByM08MvqU
249,seni görmek çok güzel,1056.117980,1059.320606,merhaba canım nasılsın o gün seni görmek çok g...,Rm6zwwnAVeA
250,seni görmek çok güzel,1097.632088,1101.225604,abone ol bu okey ses geldi tamam süper evet ar...,AvfpnsqaWw4


In [74]:
df_word_group_time_loc.start_time = df_word_group_time_loc.start_time.apply(lambda x: round(x))
df_word_group_time_loc.end_time = df_word_group_time_loc.end_time.apply(lambda x: round(x))
df_word_group_time_loc 

,search_string,start_time,end_time,sentence,video_id
0,bana yardım eder misin,2875,2878,teyzeciğim ne olur bana yardım eder misin,QIF4PAQBRS8
1,bana yardım eder misin,3497,3500,bana yardım eder misin,Qmn1S9yjxB4
2,bana yardım eder misin,4624,4627,bana yardım eder misin,SnbDtuHGdqE
3,bana yardım eder misin,78,80,şu oyuncağı bir türlü kaldıramıyorum bana yard...,LMHUJ60Fsho
4,bana yardım eder misin,922,926,ya bana yardım eder misin,OBoAMIqiTWU
...,...,...,...,...,...
247,seni görmek çok güzel,2447,2450,yaşayabilir akımı ben yılmaz merhaba yılmaz se...,cUAQrHbbpZ8
248,seni görmek çok güzel,6004,6007,sonra yumarım açarsın kontrol sızladı da merha...,1iByM08MvqU
249,seni görmek çok güzel,1056,1059,merhaba canım nasılsın o gün seni görmek çok g...,Rm6zwwnAVeA
250,seni görmek çok güzel,1098,1101,abone ol bu okey ses geldi tamam süper evet ar...,AvfpnsqaWw4


In [75]:
df_word_group_time_loc["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc['video_id'].map(str)+"&t="+df_word_group_time_loc['start_time'].map(str)+"s"
df_word_group_time_loc

,search_string,start_time,end_time,sentence,video_id,video_url
0,bana yardım eder misin,2875,2878,teyzeciğim ne olur bana yardım eder misin,QIF4PAQBRS8,https://www.youtube.com/watch?v=QIF4PAQBRS8&t=...
1,bana yardım eder misin,3497,3500,bana yardım eder misin,Qmn1S9yjxB4,https://www.youtube.com/watch?v=Qmn1S9yjxB4&t=...
2,bana yardım eder misin,4624,4627,bana yardım eder misin,SnbDtuHGdqE,https://www.youtube.com/watch?v=SnbDtuHGdqE&t=...
3,bana yardım eder misin,78,80,şu oyuncağı bir türlü kaldıramıyorum bana yard...,LMHUJ60Fsho,https://www.youtube.com/watch?v=LMHUJ60Fsho&t=78s
4,bana yardım eder misin,922,926,ya bana yardım eder misin,OBoAMIqiTWU,https://www.youtube.com/watch?v=OBoAMIqiTWU&t=...
...,...,...,...,...,...,...
247,seni görmek çok güzel,2447,2450,yaşayabilir akımı ben yılmaz merhaba yılmaz se...,cUAQrHbbpZ8,https://www.youtube.com/watch?v=cUAQrHbbpZ8&t=...
248,seni görmek çok güzel,6004,6007,sonra yumarım açarsın kontrol sızladı da merha...,1iByM08MvqU,https://www.youtube.com/watch?v=1iByM08MvqU&t=...
249,seni görmek çok güzel,1056,1059,merhaba canım nasılsın o gün seni görmek çok g...,Rm6zwwnAVeA,https://www.youtube.com/watch?v=Rm6zwwnAVeA&t=...
250,seni görmek çok güzel,1098,1101,abone ol bu okey ses geldi tamam süper evet ar...,AvfpnsqaWw4,https://www.youtube.com/watch?v=AvfpnsqaWw4&t=...


In [76]:
word_count_result(df_word_group_time_loc, ["search_string"], set_condition=True)

,word,word_count
0,şey,4
1,ne,4
2,de,4
3,daha,4
4,bir,4
...,...,...
56,yapmak,1
57,sakin,1
58,bana,1
59,olduğunu,1


In [77]:
df_word_group_time_loc.to_excel(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result_{file_ext}.xlsx", index=False) 

#### Copy Move And Delete

In [78]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group_With_{word_end}_Word_{sample_num}_Youtube_{time_shift}s_Timeshift_For_Talk_Time_Result_{file_ext}.xlsx")
output_file

['Turkish_Word_Group_With_1000_Word_10_Youtube_0.6s_Timeshift_For_Talk_Time_Result_Fourgram.xlsx']

In [79]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [80]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass